In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
import plotly.graph_objects as go
import plotly.express as px
# importing the native rouge library
from rouge_score import rouge_scorer
from transformers import T5ForConditionalGeneration, T5Tokenizer
from tensorflow.keras.models import load_model
from pyvi import ViTokenizer, ViPosTagger # thư viện NLP tiếng Việt
from tqdm import tqdm
import gensim # thư viện NLP
import requests
from bs4 import BeautifulSoup
#import thư viện 
from numpy import genfromtxt
from underthesea import sent_tokenize
from underthesea import word_tokenize
import matplotlib.pyplot as plt
import re

In [6]:
data = pd.read_csv('dataset_crawl2.csv')

In [7]:
data.head()

,category_sentence,title,description,Sentence
0,Thời sự,Tin sáng 28-9: Giá rau tăng do mưa bão; TP.HCM...,TTO - Giải ngân vốn đầu tư nước ngoài tiếp tục...,"['', 'Do mưa bão, giá nhiều loại rau Đà Lạt nh..."
1,Thế giới,Nga 'yêu cầu' Tổng thống Ukraine phát biểu trự...,"TTO - Ngày 27-9, Đại sứ Nga tại Liên Hiệp Quốc...","['', 'Đại sứ Nga tại Liên Hiệp Quốc Vasily Neb..."
2,Bạn đọc làm báo,Che chở nhau qua siêu bão,TTO - Chương trình có nhiều thông tin cập nhật...,"['', 'Cụ Nguyễn Bá Loan (95 tuổi, thôn Tân Hy,..."
3,Thể thao,"Sau hai trận thua, mỗi trận vô ba trái, tuyển ...","TTO - Tối 26-9, có vẻ như mọi sự chú ý dồn về ...","['', 'Văn Toàn (9) được đồng đội chia vui sau ..."
4,Kinh doanh,"'Nếu cho tư nhân quản lý nhà ga hàng không, sẽ...",TTO - Khách quốc tế chưa phục hồi hoàn toàn nh...,"['', 'Nhiều thời điểm tại Tân Sơn Nhất, khách ..."


In [8]:
def text_preprocess(sentence):
    sentence = word_tokenize(sentence, format="text")
    sentence = re.sub(r"[-()\"#/@;:<>{}`+=~|*'.!?,.]", "", sentence)
    sentence = re.sub(r'[^\s\wáàảãạăắằẳẵặâấầẩẫậéèẻẽẹêếềểễệóòỏõọôốồổỗộơớờởỡợíìỉĩịúùủũụưứừửữựýỳỷỹỵđ]',' ',sentence)
    sentence.lower()
    sentence = re.sub(r'\s+', ' ', sentence).strip()
    return sentence

In [9]:
stopword = ["ăn", "nhưng", "mình", "có", "là", "thì", "và", "cũng", "mà", "ở", "nên", "lại", "thấy", "đây", "đi", "như", "ra", "cho", "với", "đến", "gọi", "còn", "cái", "lần", "phải", "vụ", "các", "của", "để", "chỉ", "mới", "luôn", "gì", "vì", "một", "rồi", "sẽ", "nào", "nữa", "đã", "lúc", "làm", "chỗ", "khi", "bị", "về", "đó", "mấy", "hay", "vẫn", "từ", "thôi", "sau", "hôm", "bên", "chung", "chắc", "lượng", "giờ", "kiểu", "xong", "thể", "chứ", "thế", "độ", "bảo", "loại", "ngoài", "vậy", "sự", "sao", "tto"]

In [10]:
def remove_stopwords(line):
    words = []
    for word in line.strip().split():
        if word not in stopword:
            words.append(word)
    return ' '.join(words)

In [11]:
def clean(reviews):
    review = []
    for sentence in reviews:
        review.append(text_preprocess(sentence))
        
    review = [i.lower() for i in review]
    
    new_review = []
    for line in review:
        new_review.append(remove_stopwords(line))
    return new_review 
    

In [12]:
Sentence = clean(data['Sentence'])
description = clean(data['description'])
#content = data['content']
catogory = data['category_sentence']
#description = data['description']

In [13]:
def st(Sentence, catogory):
    du_lieu = pd.DataFrame()
    du_lieu['Sentence'] = Sentence
    du_lieu['category_sentence'] = catogory
    df = pd.read_csv('data1.csv')
    data2 = df.append(du_lieu, ignore_index = True)
    X_data = data2['Sentence']
    y_data = data2['category_sentence']
    with open('vietnamese-stopwords-dash.txt', 'r', encoding="utf8") as f:
        stopwords = set([w.strip() for w in f.readlines()])
        
    for i in range(len(X_data)):
        text = X_data[i]
        try:
            split_words =  [x.strip('0123456789%@$.,=+-!;/()*"&^:#|\n\t\'').lower() for x in text.split()]
        except TypeError:
            split_words =  []
        X_data[i] = ' '.join([word for word in split_words if word not in stopwords])
        
    
    
    return X_data, y_data
    

In [14]:
st = st(Sentence, catogory)
st[1]

<ipython-input-13-d01fb54e76e9>:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_data[i] = ' '.join([word for word in split_words if word not in stopwords])


0        Chinh tri Xa hoi
1        Chinh tri Xa hoi
2        Chinh tri Xa hoi
3        Chinh tri Xa hoi
4        Chinh tri Xa hoi
               ...       
50388            Thể thao
50389            Thế giới
50390             Thời sự
50391             Thời sự
50392             Thời sự
Name: category_sentence, Length: 50393, dtype: object

In [15]:
from sklearn.model_selection import train_test_split
x, X_test, y, y_test = train_test_split(st[0], st[1], test_size=0.2)
X_train, X_val, y_train, y_val = train_test_split(x, y, test_size=0.3)

In [16]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vect = TfidfVectorizer(analyzer='word', max_features=10000)
#Tính tfidf cho tập train
tfidf_vect.fit(st[0])
#pickle.dump(tfidf_vect, open("/content/drive/My Drive/NLP/Model/vectorizer.pickle", "wb"))
x_trainn =  tfidf_vect.transform(X_train)
x_testt =  tfidf_vect.transform(X_test)
x_vall =  tfidf_vect.transform(X_val)
tfidf_X_data = tfidf_vect.transform(st[0])

In [17]:
from sklearn.decomposition import TruncatedSVD

svd = TruncatedSVD(n_components=500, random_state=1998)
svd.fit(tfidf_X_data)
#svd.fit(X_data)
#pickle.dump(svd, open("/content/drive/My Drive/NLP/Model/selector.pickle", "wb"))

x_train = svd.transform(x_trainn)
x_test = svd.transform(x_testt)
x_val = svd.transform(x_vall)

In [18]:
def classification(sentence):
    model1 = load_model('classification.h5')
    
    dt = [sentence] 

    dtt = tfidf_vect.transform(dt)
    
    dttt = svd.transform(dtt)
    
    l = model1.predict(dttt)
    maximun = np.max(l)
    
    maxx = np.argmax(l)
    classification = []
    if maxx == 0:
        classification = 'Chính trị xã hội'
    elif maxx == 1:
        classification = 'Đời sống'
    elif maxx == 2:
        classification = 'Kinh doanh'
    elif maxx == 3:
        classification = 'Khoa học'
    elif maxx == 4:
        classification = 'Pháp luật'
    elif maxx == 5:
        classification = 'Sức khỏe'
    elif maxx == 6:
        classification = 'Thế giới'
    elif maxx == 7:
        classification = 'Thể thao'
    elif maxx == 8:
        classification = 'Văn hóa'
    else:
        classification = 'Vi tính'
        
    return classification, maximun
        
    

In [19]:
def summarization(sentence):
    path = 'model_summarization'
    model = T5ForConditionalGeneration.from_pretrained(path)
    tokenizer = T5Tokenizer.from_pretrained(path)
    
    tokenized_texts = tokenizer.encode(sentence, return_tensors="pt").to(device)
    summary_ids = model.generate(
                    tokenized_texts,
                    max_length=256, 
                    num_beams=5,
                    repetition_penalty=2.5, 
                    length_penalty=1.0, 
                    early_stopping=True
                )
    output = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    
    return output

    

In [20]:
def rouge_scores(auto_sum, description):
    scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
    scores = scorer.score(auto_sum, description)
    
    kq = pd.DataFrame(scores, index=('precision', 'recall', 'f-measure'))
    
    kq2 = kq.T
    
    return kq2
    

In [21]:
if torch.cuda.is_available():       
    device = torch.device("cuda")

    print('There are %d GPU(s) available.' % torch.cuda.device_count())

    print('We will use the GPU:', torch.cuda.get_device_name(0))
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

No GPU available, using the CPU instead.


# Classification

In [79]:
data1 = pd.read_csv('dataset_crawl1.csv')

In [80]:
data1

,category_sentence,title,description,Sentence
0,Thế giới,Quốc vụ khanh Ấn Độ: TP.HCM phát triển nhanh b...,TTO - Quốc vụ khanh Bộ Ngoại giao Ấn Độ Rajkum...,"['', 'Quốc vụ khanh Bộ Ngoại giao Ấn Độ Rajkum..."
1,Giải trí,"Thấy gì từ việc phim Thái, Hàn chiếm lĩnh rạp ...",TTO - Không phải phim bom tấn đắt tiền nhưng p...,"['', 'Trailer phim ""Bỗng dưng trúng số""', 'Phi..."
2,Thời sự,"Gần 800 ngư dân vào các âu tàu, làng chài ở...","TTO - Đến trưa 27-9, có gần 800 ngư dân củ...","['', 'Tàu cá vào tránh bão số 4 ở âu ta..."
3,Pháp luật,Thanh tra Chính phủ chọn ngẫu nhiên 30 cán bộ ...,TTO - Thanh tra Chính phủ sẽ lựa chọn ngẫu nhi...,"['', 'Thanh tra Chính phủ vừa ban hành kế hoạc..."
4,Pháp luật,Công an TP.HCM bắt nhóm giả nhân viên ngân hàn...,TTO - Công an vừa triệt phá băng nhóm chuyên g...,"['', 'Nguyễn Quốc Đạt khai nhận hành vi lừa đả..."
5,Kinh doanh,Công ty ông Dũng “lò vôi” phủ nhận bán một phầ...,TTO - Trước thông tin lan truyền cho rằng một ...,"['', 'Ông Huỳnh Uy Dũng hiện vẫn điều hành trự..."
6,Giáo dục,Những khoản thu gây choáng đầu năm học: Tiền đ...,"TTO - Trong hai ngày 24, 25-9 nhiều trường phổ...","['', '', 'Có thể nói tiền điểm danh, tổ chức k..."
7,Giải trí,Khi trang phục dân tộc được sáng tạo quá đà,TTO - Phần thi trang phục dân tộc luôn nhận đư...,"['', 'Sự kết hợp giữa áo yếm và áo bà ba gây t..."
8,Kinh doanh,VCCI: Có thể miễn toàn bộ thuế tiêu thụ đặc bi...,TTO - Góp ý dự thảo nghị quyết của Quốc hội về...,"['', 'Người dân đổ xăng trên đường Điện Biên P..."
9,Xe,Các tính năng thú vị trên ô tô nhưng ít ai biế...,"Một số tính năng ô tô mang tính thực dụng, tro...","['', 'Nhiều tính năng ô tô thú vị đã bị bỏ qua..."


In [81]:
Sentence = clean(data1['Sentence'])
description = clean(data1['description'])
catogory = data1['category_sentence']

In [82]:
print(Sentence[4])

nguyễn_quốc_đạt khai nhận hành_vi lừa_đảo tại cơ_quan công_an ảnh công_an cung_cấp sáng 279 xa0công an huyện bình_chánh tphcm tạm giữ lê_thị_thanh_sáu 32 tuổi xa0ngụ quận tân_phú cầm_đầu xa0và nguyễn_quốc_đạt 23 tuổi ngụ xa0tỉnh đồng_nai cùng 6 người khác xa0có liên_quan điều_tra hành_vi lừa_đảo chiếm_đoạt tài_sản sáu đạt khai nhận cả hai thỏa_thuận cùng nhau điều_hành nhóm lừa_đảo phân_công công_việc rõ_ràng ăn_chia lợi_nhuận phương_thức hoạt_động nhóm này giả_danh_nhân_viên ngân_hàng điện_thoại hại theo danh_sách dữ_liệu khách_hàng những người này xa0tìm mua được trên trang mạng tư_vấn gói vay lãi_suất 0 đồng trước nhận được khoản vay theo nhu_cầu hại đóng phí bảo_hiểm nhóm này số tiền 18 triệu đồng xa0cho gói vay 10 50 triệu hoặc 38 triệu đồng xa0cho gói vay 60 100 triệu đồng nhận được thẻ ngân_hàng không rút được tiền hại liên_lạc đối_tượng phản_hồi do thẻ lỗi cử_nhân_viên tận nhà giao tiền trực_tiếp cắt đứt liên_lạc hại đối_tượng sáu khai nhận bắt_đầu hoạt_động lừa_đảo nói trên và

In [83]:
print(catogory[4])

Pháp luật


In [84]:
classification(Sentence[4])

('Pháp luật', 0.9992455)

# Summarization

In [85]:
print(Sentence[4])

nguyễn_quốc_đạt khai nhận hành_vi lừa_đảo tại cơ_quan công_an ảnh công_an cung_cấp sáng 279 xa0công an huyện bình_chánh tphcm tạm giữ lê_thị_thanh_sáu 32 tuổi xa0ngụ quận tân_phú cầm_đầu xa0và nguyễn_quốc_đạt 23 tuổi ngụ xa0tỉnh đồng_nai cùng 6 người khác xa0có liên_quan điều_tra hành_vi lừa_đảo chiếm_đoạt tài_sản sáu đạt khai nhận cả hai thỏa_thuận cùng nhau điều_hành nhóm lừa_đảo phân_công công_việc rõ_ràng ăn_chia lợi_nhuận phương_thức hoạt_động nhóm này giả_danh_nhân_viên ngân_hàng điện_thoại hại theo danh_sách dữ_liệu khách_hàng những người này xa0tìm mua được trên trang mạng tư_vấn gói vay lãi_suất 0 đồng trước nhận được khoản vay theo nhu_cầu hại đóng phí bảo_hiểm nhóm này số tiền 18 triệu đồng xa0cho gói vay 10 50 triệu hoặc 38 triệu đồng xa0cho gói vay 60 100 triệu đồng nhận được thẻ ngân_hàng không rút được tiền hại liên_lạc đối_tượng phản_hồi do thẻ lỗi cử_nhân_viên tận nhà giao tiền trực_tiếp cắt đứt liên_lạc hại đối_tượng sáu khai nhận bắt_đầu hoạt_động lừa_đảo nói trên và

In [86]:
print(description[4])

công_an vừa triệt_phá băng_nhóm chuyên giả_danh_nhân_viên ngân_hàng điện_thoại hại theo danh_sách dữ_liệu khách_hàng vay lãi_suất 0 đồng nhóm này bắt nạn_nhân đóng tiền phí bảo_hiểm bặt vô âm_tín


In [87]:
auto_sum = summarization(Sentence[4])
auto_sum

'Công an huyện bình_chánh tạm giữ lê_thị thanh_sáu 32 tuổi xa0ngụ quận tân_phú cầm_đầu xa0và nguyễn_quốc_đạt 23 tuổi ngụ xa0tỉnh đồng_nai cùng 6 người khác xa0có liên quan điều tra hành vi lừa_đảo chiếm_đoạt hàng trăm triệu đồng.'

In [88]:
rouge_score = rouge_scores(auto_sum, description[16])
rouge_score

,precision,recall,f-measure
rouge1,0.480000,0.486486,0.483221
rouge2,0.081081,0.082192,0.081633
rougeL,0.226667,0.229730,0.228188


In [89]:
fig = px.bar(rouge_score, barmode='group', range_y=[0, 1], 
            labels={'x':'Phương pháp đánh giá', 'y':'Độ chính xác'},
             title='Đánh giá văn bản tóm tắt'
            )
fig.show()